<a href="https://colab.research.google.com/github/appliedcode/mthree-c422/blob/mthree-422-salleh/Exercises/day-10/Audio_Classification_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement: Audio Classification with Transformers on the ESC-50 Environmental Sound Dataset

This is from the file Exercises/day-10/Problem_Statement_Audio.md

### Objective

Build and evaluate a transformer-based audio classification model using the ESC-50 dataset. The task involves loading and preprocessing environmental audio clips, applying a pretrained transformer or fine-tuning it for classifying audio into 50 environmental sound classes, and analyzing model performance.

### Dataset

**ESC-50** (Environmental Sound Classification)

- Publicly available dataset of 2,000 labeled environmental audio recordings (5 seconds each) across 50 classes such as dog bark, rain, siren, and keyboard typing.
- Download URL: https://github.com/karolpiczak/ESC-50 (direct audio files in WAV format and metadata CSV)
- The dataset is well suited for benchmarking environmental audio classification models.


### Learning Objectives

- Load and preprocess short environmental audio clips (sample rate standardization, normalization).
- Extract audio features suitable for transformer models (raw waveform or learned embeddings).
- Use a pretrained audio transformer model (e.g., Wav2Vec 2.0, Hubert) as a feature extractor or fine-tune it for classification.
- Train, validate, and test the model on the ESC-50 dataset classes.
- Visualize training progress and assess model accuracy, confusion matrices, and class-wise performance.
- Investigate the attention patterns of the transformer to understand which audio segments influence classification.


### Tasks

1. **Dataset Preparation**
    - Download and unzip ESC-50.
    - Load metadata CSV to map audio file names to labels.
    - Implement audio loading pipeline (e.g., using librosa or torchaudio) to read and preprocess the clips uniformly.
2. **Feature Extraction and Modeling**
    - Use pretrained transformer audio models (e.g., `facebook/wav2vec2-base`, `superb/hubert-large-superb-ks`) from Hugging Face Transformers.
    - Convert audio clips into model inputs (waveform arrays or processed features).
    - Fine-tune on ESC-50 or extract fixed embeddings and train a classifier head.
3. **Training and Evaluation**
    - Split data into training and testing sets as per standard ESC-50 folds or create your own split.
    - Train the model and monitor learning curves.
    - Evaluate accuracy, per-class precision \& recall, and generate confusion matrix.
4. **Attention Visualization and Analysis (Optional)**
    - Visualize attention maps or gradients to interpret model focus on specific temporal audio segments.
    - Analyze attention differences across sound classes.
5. **Reporting**
    - Summarize data preprocessing challenges and solutions.
    - Present quantitative model results and qualitative insights from attention visualizations.
    - Discuss potential improvements like data augmentation or ensembling.

### Deliverables

- Python notebook or script implementing data loading, preprocessing, training, and evaluation.
- Visualizations of attention weights and model performance metrics.
- A brief report or markdown summarizing findings, challenges, and future directions.

This problem encourages hands-on experience with transformer models for audio classification on a real, publicly available dataset, expanding beyond speech recognition to environmental sound understanding.

If you'd like, I can also help generate starter code or a complete Colab notebook for this task! Would you be interested?

In [1]:
# Install required packages
!pip install transformers datasets torch torchaudio librosa soundfile scikit-learn matplotlib seaborn

import os
import pandas as pd
import numpy as np
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model
import requests
import zipfile

import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device: {device}")

Using device: cpu


In [ ]:
# Download the ESC-50 dataset - To complete